In [106]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt


from qiskit import Aer, execute

from qiskit_ionq_provider import IonQProvider 

#Call provider and set token value
provider = IonQProvider(token='BFmvdArkiCbsS12r4LZf5VgYDo4HngsS')

In [107]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7ff68904ee90>()>,
 <IonQQPUBackend('ionq_qpu') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7ff68904ee90>()>]

In [108]:
from qiskit import QuantumCircuit
from random import randint

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import RYGate
#c3h_gate = RYGate(np.pi).control(2, ctrl_state='00')
#qc.append(c3h_gate, qr)


def crypto_hash_init(bitstring, K):

    d = len(K)
    n = len(bitstring)
    big_N = 2**n

    qr = QuantumRegister(d)
    qc_hash = QuantumCircuit(qr)
    
    return (qc_hash, qr)
    

In [109]:
def crypto_hash_add_H(bitstring, K, qc_hash):
    for i in range(d-1):
        qc_hash.h(i)
    
    return qc_hash

In [110]:
def crypto_hash_final(bitstring, K, qc_hash, qr):
    def dec_to_bin(x,d):
        output = str(int(bin(x)[2:]))
        if len(output) < d-1:
            zeros = ("0" * (d-1-len(output)))
            return zeros + output
        else:
            return output

    for n_index in range(1,n+1):
        control_states = "0" * (d-1)
        if int(bitstring[n_index-1]) == 1:
            for perm in range(0,2**(d-1)):     
                control_states = dec_to_bin(perm,d)            
                i = sum(int(c) for c in control_states.strip())
                Y_Rotate_Gate = RYGate(4*np.pi*K[i-1]*2**n_index/big_N).control(2, ctrl_state=control_states)
                qc_hash.compose(Y_Rotate_Gate, qr, inplace=True)
        
    return qc_hash


In [111]:
def crypto_hash(bitstring, K):
    (qc, qr) = crypto_hash_init(bitstring, K)
    qc = crypto_hash_add_H(bitstring, K, qc)
    qc_hash = crypto_hash_final(bitstring, K, qc, qr)
    
    backend = Aer.get_backend('statevector_simulator')
    state = execute(qc,backend).result().get_statevector()
    return(state)

